In [5]:
import os
import json
import cv2
from tqdm import tqdm
from collections import defaultdict

gt_dir = '/opt/ml/input/data/ICDAR17/ICDAR17_gt_txt'
img_dir = '/opt/ml/input/data/ICDAR17/images'

In [2]:
img_names = os.listdir(img_dir)
img_names = [img_name for img_name in img_names if not img_name.startswith('.')]
img_names = [img_name for img_name in img_names if img_name.split('.')[-1]!='gif'] # gif 제외
img_names.sort(key=lambda x:int(x.split('.')[0].split('_')[-1]))
img_names[:15], len(img_names)

['img_1.png',
 'img_2.png',
 'img_3.jpg',
 'img_4.jpg',
 'img_5.jpg',
 'img_6.jpg',
 'img_7.jpg',
 'img_8.jpg',
 'img_9.jpg',
 'img_10.jpg',
 'img_11.jpg',
 'img_12.jpg',
 'img_13.jpg',
 'img_14.jpg',
 'img_15.jpg',
 'img_16.jpg',
 'img_17.jpg',
 'img_18.jpg',
 'img_19.jpg',
 'img_20.jpg',
 'img_21.jpg',
 'img_22.jpg',
 'img_23.jpg',
 'img_24.jpg',
 'img_25.jpg',
 'img_26.jpg',
 'img_27.jpg',
 'img_28.jpg',
 'img_29.jpg',
 'img_30.jpg',
 'img_31.jpg',
 'img_32.jpg',
 'img_33.jpg',
 'img_34.jpg',
 'img_35.jpg',
 'img_36.jpg',
 'img_37.jpg',
 'img_38.jpg',
 'img_39.jpg',
 'img_40.jpg',
 'img_41.jpg',
 'img_42.jpg',
 'img_43.jpg',
 'img_44.jpg',
 'img_45.jpg',
 'img_46.jpg',
 'img_47.jpg',
 'img_48.jpg',
 'img_49.jpg',
 'img_50.jpg',
 'img_51.jpg',
 'img_52.jpg',
 'img_53.jpg',
 'img_54.jpg',
 'img_55.jpg',
 'img_56.jpg',
 'img_57.jpg',
 'img_58.jpg',
 'img_59.jpg',
 'img_60.jpg',
 'img_61.jpg',
 'img_62.jpg',
 'img_63.jpg',
 'img_64.jpg',
 'img_65.jpg',
 'img_66.jpg',
 'img_67.jpg',
 'im

In [ ]:
lang = defaultdict(set)

In [6]:
# 저장할 json 객체 생성
new_json = {}
new_json['images'] = {}
new_json['version'] = {}
new_json['tags'] = {}

for img_name in tqdm(img_names):
#for img_name in tqdm(lang['Korean']|lang['Latin']):
    img = cv2.imread(os.path.join(img_dir,img_name))
    img_h = int(img.shape[0])
    img_w = int(img.shape[1])
    gt_name = 'gt_' + img_name.split('.')[0] + '.txt' # 이미지에서 gt 파일명 생성 17용
    #gt_name = img_name.split('.')[0] + '.txt' # 이미지에서 gt 파일명 생성 19용

    num = int(img_name.split('.')[0].split('_')[-1])
    #if num%1000 > 800: # valid <=, train >
    #    continue

    new_json['images'][img_name] = {}

    new_json['images'][img_name]['paragraphs'] = None
    new_json['images'][img_name]['words'] = {}
    new_json['images'][img_name]['chars'] = None
    new_json['images'][img_name]['img_w'] = img_w
    new_json['images'][img_name]['img_h'] = img_h
    new_json['images'][img_name]['tags'] = None
    new_json['images'][img_name]['relations'] = None
    new_json['images'][img_name]['annotation_los'] = None
    new_json['images'][img_name]['license_tag'] = None

    f = open(os.path.join(gt_dir,gt_name),'r')
    lines = [list(line.strip().split(',')) for line in f.readlines()]
    f.close()
    for i,line in enumerate(lines):
        if len(line)>10:
            lines[i] = line[:9]
            lines[i].append(','.join(line[9:]))
        elif len(line)<10:
            print(line)

    idx = 1
    for line in lines:
        x1,y1,x2,y2,x3,y3,x4,y4,language,transcription = line
        bbox_idx = '0'*(4-len(str(idx))) + str(idx)
        new_json['images'][img_name]['words'][bbox_idx] = {}
        new_json['images'][img_name]['words'][bbox_idx]['transcription'] = transcription
        new_json['images'][img_name]['words'][bbox_idx]['points'] = [[x1,y1],[x2,y2],[x3,y3],[x4,y4]]
        new_json['images'][img_name]['words'][bbox_idx]['orientation'] = 'Horizontal'
        #new_json['images'][img_name]['words'][bbox_idx]['language'] = ['ko'] if language=='Korean' else (['en'] if language=='Latin' else ['others'])
        new_json['images'][img_name]['words'][bbox_idx]['language'] = [language]
        lang[language].add(img_name)
        new_json['images'][img_name]['words'][bbox_idx]['tags'] = []
        new_json['images'][img_name]['words'][bbox_idx]['confidence'] = None
        new_json['images'][img_name]['words'][bbox_idx]['illegibility'] = transcription == '###'

        # Korean, Latin 제외 나머진 모두 dontcare
        if not(language=='Korean' or language=='Latin'):
            new_json['images'][img_name]['words'][bbox_idx]['illegibility'] = True
            new_json['images'][img_name]['words'][bbox_idx]['transcription'] = '###'


        idx += 1

    

 27%|██▋       | 1947/7196 [01:01<03:58, 22.01it/s]

In [ ]:
len(list(new_json['images'].keys()))

In [ ]:
for k,v in lang.items():
    print(k,len(v))

In [60]:
with open('/opt/ml/input/data/ICDAR17/ufo/ICDAR17_train.json','w') as f:
        json.dump(new_json, f, indent=4)